In [11]:
import sys
from itertools import combinations

In [12]:
def load_transactions(filename="browsing.txt"):
    """Read transactions from the file; each line is a session (items separated by spaces)."""
    transactions = []
    with open(filename, 'r') as f:
        for line in f:
            items = line.strip().split()
            if items:
                # Remove duplicates per session by converting to a set.
                transactions.append(set(items))
    print(f"DEBUG: Loaded {len(transactions)} transactions from {filename}")
    return transactions


In [13]:
def count_frequent_items(transactions, min_support):
    """Count items and return a dict {item: count} for items with count >= min_support."""
    item_counts = {}
    for t in transactions:
        for item in t:
            item_counts[item] = item_counts.get(item, 0) + 1
    L1 = {item: count for item, count in item_counts.items() if count >= min_support}
    return L1

In [14]:
def count_frequent_pairs(transactions, L1, min_support):
    """Generate candidate pairs from L1 and count their supports."""
    L1_items = set(L1.keys())
    pair_counts = {}
    for t in transactions:
        # Only consider items that are frequent
        t_items = sorted(t.intersection(L1_items))
        for pair in combinations(t_items, 2):
            pair_counts[pair] = pair_counts.get(pair, 0) + 1
    L2 = {pair: count for pair, count in pair_counts.items() if count >= min_support}
    return L2

In [15]:
def count_frequent_triples(transactions, L1, L2, min_support):
    """Generate candidate triples (only if all 2‑item subsets are frequent) and count supports."""
    L1_items = set(L1.keys())
    triple_counts = {}
    for t in transactions:
        t_items = sorted(t.intersection(L1_items))
        for triple in combinations(t_items, 3):
            # Prune candidate triple: all pairs must be frequent
            if ((triple[0], triple[1]) in L2 and
                (triple[0], triple[2]) in L2 and
                (triple[1], triple[2]) in L2):
                triple_counts[triple] = triple_counts.get(triple, 0) + 1
    L3 = {triple: count for triple, count in triple_counts.items() if count >= min_support}
    return L3

In [16]:
def generate_pair_rules(L1, L2):
    """
    For each frequent pair (X, Y) with support s,
    generate rules X⇒Y (confidence = s/support(X)) and Y⇒X (confidence = s/support(Y)).
    """
    pair_rules = {}
    for (i, j), support in L2.items():
        pair_rules[(i, j)] = support / L1[i]  # Rule: i ⇒ j
        pair_rules[(j, i)] = support / L1[j]  # Rule: j ⇒ i
    return pair_rules


In [17]:
def generate_triple_rules(L2, L3):
    """
    For each frequent triple (a, b, c) with support s,
    generate:
      - (a,b)⇒c (confidence = s/support({a,b}))
      - (a,c)⇒b (confidence = s/support({a,c}))
      - (b,c)⇒a (confidence = s/support({b,c}))
    """
    triple_rules = {}
    for (a, b, c), support in L3.items():
        if (a, b) in L2:
            triple_rules[((a, b), c)] = support / L2[(a, b)]
        if (a, c) in L2:
            triple_rules[((a, c), b)] = support / L2[(a, c)]
        if (b, c) in L2:
            triple_rules[((b, c), a)] = support / L2[(b, c)]
    return triple_rules


In [18]:
def rank_rules(rules):
    """
    Sort rules (dictionary where key is (lhs, rhs) and value is confidence)
    in descending order of confidence.
    In case of ties, sort by lexicographically increasing order on the left-hand side
    (and then the right-hand side).
    
    Returns a dict mapping each rule to a tuple (confidence, ranking).
    """
    def sort_key(item):
        key, conf = item
        lhs, rhs = key
        # For pair rules, ensure lhs is a tuple for proper lex order.
        lhs_key = (lhs,) if isinstance(lhs, str) else lhs
        return (-conf, lhs_key, rhs)
    
    sorted_rules = sorted(rules.items(), key=sort_key)
    ranking = {}
    for rank, (rule, conf) in enumerate(sorted_rules, start=1):
        ranking[rule] = (conf, rank)
    return ranking

In [19]:
def main(data_file, input_rule):
    min_support = 100
    transactions = load_transactions(data_file)
    
    # Pass 1: Frequent items
    L1 = count_frequent_items(transactions, min_support)
    # Pass 2: Frequent pairs (2-itemsets)
    L2 = count_frequent_pairs(transactions, L1, min_support)
    # Pass 3: Frequent triples (3-itemsets)
    L3 = count_frequent_triples(transactions, L1, L2, min_support)
    
    # Generate association rules.
    pair_rules = generate_pair_rules(L1, L2)
    triple_rules = generate_triple_rules(L2, L3)
    
    # Rank rules separately.
    pair_ranking = rank_rules(pair_rules)
    triple_ranking = rank_rules(triple_rules)
    
    # Determine if the input rule is for a pair or a triple based on its length.
    if len(input_rule) == 16:
        # Pair rule: first 8 characters = LHS, next 8 = RHS.
        lhs = input_rule[:8]
        rhs = input_rule[8:16]
        key = (lhs, rhs)
        if key in pair_ranking:
            conf, rank = pair_ranking[key]
            print(f"{conf} , {rank}")
        else:
            print("NA")
    elif len(input_rule) == 24:
        # Triple rule: first 16 characters = LHS (two 8-char items), last 8 = RHS.
        item1 = input_rule[:8]
        item2 = input_rule[8:16]
        rhs = input_rule[16:24]
        # Our triple rules use a sorted pair for the left-hand side.
        lhs = tuple(sorted([item1, item2]))
        key = (lhs, rhs)
        if key in triple_ranking:
            conf, rank = triple_ranking[key]
            print(f"{conf} , {rank}")
        else:
            print("NA")
    else:
        print("NA")

if __name__ == '__main__':
    # Since the data file is always browsing.txt, we set it here.
    data_file = "browsing.txt"
    if len(sys.argv) != 2:
        print("Usage: python apriori.py <rule>")
        sys.exit(1)
    input_rule = sys.argv[1]
    main(data_file, input_rule)

DEBUG: Loaded 31101 transactions from browsing.txt
NA
